In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report
import keras
import numpy as np
import csv
import random

Using TensorFlow backend.


In [2]:
infile = open('../input/voiceidentification/features.csv')
tests = open('../input/voiceidentification/test_indices.csv')
trains = open('../input/voiceidentification/train_indices.csv')
csvreader = csv.reader(infile)
csv1 = csv.reader(tests)
csv2 = csv.reader(trains)

data = list(csvreader)
training_data = [data[int(i[0])] for i in csv2]
testing_data = [data[int(i[0])] for i in csv1]
training_labels = np.array([[int(i[0])] for i in training_data])
testing_labels = np.array([[int(i[0])] for i in testing_data])
training_data = np.array([i[1:] for i in training_data])
testing_data = np.array([i[1:] for i in testing_data])

Y = np.concatenate((training_labels,testing_labels),axis=0)
X = np.concatenate((training_data,testing_data),axis=0)

infile.close()
tests.close()
trains.close()

In [3]:
model = Sequential([Dense(1024, activation='relu', input_shape=(len(training_data[0]),)), Dropout(0.5), Dense(256, activation='relu'), Dropout(0.5), Dense(64, activation='relu'), Dropout(0.5), Dense(10, activation='softmax')])
model.load_weights("../input/weightsafter85epochs/weights-improvement-85-0.42.hdf5")
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

one_hot_labels = keras.utils.to_categorical(Y, num_classes=10)
model.fit(X, one_hot_labels, validation_split=0.1, epochs=200, batch_size=32, callbacks=callbacks_list, verbose=1)


Train on 297212 samples, validate on 33024 samples
Epoch 1/200
297212/297212 [==============================] - 58s 195us/step - loss: 1.7443 - accuracy: 0.3782 - val_loss: 1.6482 - val_accuracy: 0.4099

Epoch 00001: val_accuracy improved from -inf to 0.40988, saving model to weights-improvement-01-0.41.hdf5
Epoch 2/200
297212/297212 [==============================] - 56s 188us/step - loss: 1.7410 - accuracy: 0.3801 - val_loss: 1.6458 - val_accuracy: 0.4077

Epoch 00002: val_accuracy did not improve from 0.40988
Epoch 3/200
297212/297212 [==============================] - 56s 187us/step - loss: 1.7447 - accuracy: 0.3785 - val_loss: 1.6506 - val_accuracy: 0.4047

Epoch 00003: val_accuracy did not improve from 0.40988
Epoch 4/200
297212/297212 [==============================] - 56s 189us/step - loss: 1.7422 - accuracy: 0.3798 - val_loss: 1.6419 - val_accuracy: 0.4143

Epoch 00004: val_accuracy improved from 0.40988 to 0.41427, saving model to weights-improvement-04-0.41.hdf5
Epoch 5/200


In [5]:
one_hot_test_labels = keras.utils.to_categorical(testing_labels, num_classes=10)
score = model.evaluate(testing_data, one_hot_test_labels)
print(score)

20000/20000 [==============================] - 2s 109us/step
[1.637090384864807, 0.41214999556541443]


In [6]:
y_pred = model.predict(testing_data)
report = classification_report(testing_labels, [np.argmax(i) for i in y_pred])

In [7]:
print(report)

              precision    recall  f1-score   support

           0       0.43      0.73      0.54      4240
           1       0.50      0.15      0.23       928
           2       0.78      0.01      0.03       502
           3       0.43      0.59      0.50      4038
           4       0.60      0.23      0.33       719
           5       0.56      0.13      0.21      1278
           6       0.31      0.41      0.35      2074
           7       0.53      0.10      0.17      1869
           8       0.38      0.37      0.37      2799
           9       0.42      0.14      0.21      1553

    accuracy                           0.41     20000
   macro avg       0.49      0.29      0.29     20000
weighted avg       0.44      0.41      0.37     20000

